In [1]:
import pandas as pd
import numpy as np
import shutil
import sys
import os
import json

In [2]:
is_vae = False

In [3]:
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')

In [4]:
embeddings = pd.read_csv(os.path.join(PARSE_DATA_DIR, 'cdae_embeddings_200.csv'))

In [5]:
embeddings.head()

,uid,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199
0,1,-1.903606e-32,6.126373e-32,-1.041586e-32,1.263656e-32,-2.556891e-32,1.446864e-32,-9.646448e-33,-5.381662e-32,6.522116e-32,...,-4.920046e-32,6.060555e-32,2.235629e-32,-4.628364e-32,2.501638e-32,2.035116e-32,-6.307185e-33,-5.740002e-32,-4.941461e-33,5.526963e-33
1,2,1.900094e-32,-1.379075e-32,1.011266e-33,2.059139e-32,1.075434e-32,-2.070337e-32,-1.327672e-32,1.270083e-32,4.228601e-32,...,6.549046e-32,-1.636893e-32,-7.016262e-32,5.906431e-32,3.564026e-32,-1.236979e-32,4.464314e-32,5.423416e-32,1.168587e-32,3.916058e-32
2,3,2.444368e-32,3.962619e-32,9.056878e-33,3.192449e-33,3.738563e-32,-1.772403e-32,-1.301265e-32,-9.639824e-33,-1.116018e-32,...,1.384151e-32,1.820109e-32,1.231361e-32,-5.183509e-32,-6.564572e-32,1.134003e-32,-1.574783e-32,1.704961e-32,2.476006e-32,-1.758963e-32
3,4,4.466596e-33,-2.635078e-32,2.006423e-32,-1.423771e-32,-6.095561e-33,6.230152e-32,2.212825e-32,1.006452e-32,-2.705935e-33,...,3.068422e-33,-2.434829e-32,1.528099e-32,-1.850326e-32,-2.335937e-32,-2.447797e-32,-6.060576e-32,-1.452331e-32,1.337930e-32,1.191742e-32
4,5,-5.134871e-32,-5.446336e-32,-2.477626e-32,-4.326088e-32,3.837729e-32,-5.030845e-32,1.458891e-32,-1.001956e-33,-5.085300e-33,...,-1.141153e-32,-5.578014e-32,-1.985515e-32,-6.105519e-32,-1.097291e-32,-6.064440e-33,-1.576612e-32,1.427874e-32,-4.925032e-32,-1.422201e-33


In [6]:
embeddings.shape

(129757, 201)

In [7]:
results = pd.read_csv(os.path.join(DATA_DIR, 'total_results_k_20.csv'))

In [8]:
results.head()

,uid,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,knn_cos_ndcg,knn_tfidf_ndcg,knn_bm25_ndcg
0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,1.0,0.746604,0.000000,0.0,0.373071,0.647699,0.500833,0.0
2,2.0,0.000000,0.247014,0.0,0.333333,0.000000,0.000000,0.0
3,3.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,4.0,0.272511,0.235409,0.0,0.371962,0.245902,0.191138,0.0


In [9]:
results.shape

(129757, 8)

In [10]:
#We are not going to use the knn, because only about 10% 
results = results.drop('knn_tfidf_ndcg', 1)
results = results.drop('knn_cos_ndcg', 1)
results = results.drop('knn_bm25_ndcg', 1)
results['zeroes'] = 0.000001

In [11]:
results.head()

,uid,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,zeroes
0,0.0,0.000000,0.000000,0.0,0.000000,0.000001
1,1.0,0.746604,0.000000,0.0,0.373071,0.000001
2,2.0,0.000000,0.247014,0.0,0.333333,0.000001
3,3.0,0.000000,0.000000,0.0,0.000000,0.000001
4,4.0,0.272511,0.235409,0.0,0.371962,0.000001


In [41]:
results

In [12]:
#Order Algorithms and keep the uid
uid = pd.DataFrame(data=results.uid)
results = results.drop('uid', 1)
pre_metadaset_dict = dict([(r[0], list(results.columns[np.argsort(list(r)[1::])])) \
    for r in list(results.to_records())])

pre_metadaset = pd.DataFrame.from_dict(pre_metadaset_dict,orient = 'index',columns=['fifth_place', 'fourth_place', 'third_place','second_place','first_place'])
pre_metadaset = uid.merge(pre_metadaset, left_index=True, right_index=True)

In [13]:
pre_metadaset.head()

,uid,fifth_place,fourth_place,third_place,second_place,first_place
0,0.0,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,zeroes
1,1.0,bpr_ndcg,lmf_ndcg,zeroes,most_popular_ndcg,als_ndcg
2,2.0,als_ndcg,lmf_ndcg,zeroes,bpr_ndcg,most_popular_ndcg
3,3.0,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,zeroes
4,4.0,lmf_ndcg,zeroes,bpr_ndcg,als_ndcg,most_popular_ndcg


In [15]:
with open(os.path.join(DATA_DIR, 'map_user.json')) as f:
    map_user = json.load(f)
    map_user_dict = {'original_id': list(map_user.keys()), 'mapped_id':list(map_user.values()) }
    map_user_df = pd.DataFrame(map_user_dict)

In [16]:
# Get the original id per result
pre_metadaset_original_users = pd.merge(left=map_user_df, right=pre_metadaset, left_on='mapped_id', right_on='uid')
# Make the original id column int64
pre_metadaset_original_users['original_id'] = pre_metadaset_original_users['original_id'].astype(np.int64)
#clean unused columns 
pre_metadaset_original_users = pre_metadaset_original_users.drop('mapped_id', 1)
pre_metadaset_original_users = pre_metadaset_original_users.drop('uid', 1)
pre_metadaset_original_users = pre_metadaset_original_users.drop('fifth_place', 1)
pre_metadaset_original_users = pre_metadaset_original_users.drop('fourth_place', 1)
pre_metadaset_original_users = pre_metadaset_original_users.drop('third_place', 1)
pre_metadaset_original_users = pre_metadaset_original_users.drop('second_place', 1)

In [17]:
pre_metadaset_original_users.head()

,original_id,first_place
0,1,zeroes
1,2,als_ndcg
2,3,most_popular_ndcg
3,4,zeroes
4,5,most_popular_ndcg


In [18]:
pre_metadaset_original_users.shape

(129757, 2)

In [91]:
pre_metadaset_original_users.loc[pre_metadaset_original_users['original_id'] == '7648']

,original_id,first_place
7124,7648,als_ndcg


In [16]:
embeddings.head()

,uid,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199
0,1,-1.903606e-32,6.126373e-32,-1.041586e-32,1.263656e-32,-2.556891e-32,1.446864e-32,-9.646448e-33,-5.381662e-32,6.522116e-32,...,-4.920046e-32,6.060555e-32,2.235629e-32,-4.628364e-32,2.501638e-32,2.035116e-32,-6.307185e-33,-5.740002e-32,-4.941461e-33,5.526963e-33
1,2,1.900094e-32,-1.379075e-32,1.011266e-33,2.059139e-32,1.075434e-32,-2.070337e-32,-1.327672e-32,1.270083e-32,4.228601e-32,...,6.549046e-32,-1.636893e-32,-7.016262e-32,5.906431e-32,3.564026e-32,-1.236979e-32,4.464314e-32,5.423416e-32,1.168587e-32,3.916058e-32
2,3,2.444368e-32,3.962619e-32,9.056878e-33,3.192449e-33,3.738563e-32,-1.772403e-32,-1.301265e-32,-9.639824e-33,-1.116018e-32,...,1.384151e-32,1.820109e-32,1.231361e-32,-5.183509e-32,-6.564572e-32,1.134003e-32,-1.574783e-32,1.704961e-32,2.476006e-32,-1.758963e-32
3,4,4.466596e-33,-2.635078e-32,2.006423e-32,-1.423771e-32,-6.095561e-33,6.230152e-32,2.212825e-32,1.006452e-32,-2.705935e-33,...,3.068422e-33,-2.434829e-32,1.528099e-32,-1.850326e-32,-2.335937e-32,-2.447797e-32,-6.060576e-32,-1.452331e-32,1.337930e-32,1.191742e-32
4,5,-5.134871e-32,-5.446336e-32,-2.477626e-32,-4.326088e-32,3.837729e-32,-5.030845e-32,1.458891e-32,-1.001956e-33,-5.085300e-33,...,-1.141153e-32,-5.578014e-32,-1.985515e-32,-6.105519e-32,-1.097291e-32,-6.064440e-33,-1.576612e-32,1.427874e-32,-4.925032e-32,-1.422201e-33


In [20]:
metadaset = pd.merge(left=embeddings, right=pre_metadaset_original_users, left_on='uid', right_on='original_id')

#Clean id column
metadaset = metadaset.drop('original_id', 1)

#To dont forget we will save the original id of user
metadaset = metadaset.rename(columns={"uid": "original_id"})

In [22]:
metadaset.shape

(129757, 202)

In [23]:
#We will save the meta_dataset
metadaset.to_csv(os.path.join(PARSE_DATA_DIR, '200_fac_metadataset_k_20.csv'), index=False)

,original_id,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,first_place
0,7648,-0.431028,0.273519,0.000467,-0.021366,0.411798,0.445795,1.206461,0.437548,-0.670044,...,1.261934,0.238986,0.150137,-0.024725,-0.349221,3.509393,-0.146421,1.508804,-0.859744,als_ndcg
1,10208,0.351687,-0.626017,0.034143,-0.080265,0.101129,-0.229498,0.261393,-1.189405,-0.451783,...,0.964082,0.825656,0.064932,0.042978,-1.043557,0.562128,0.234560,0.813003,0.511105,zeroes
2,13315,-0.706174,-0.031290,-0.005091,-0.097111,0.111871,0.484369,0.737223,-1.132764,1.119018,...,1.040254,-1.375516,-0.064951,0.030435,0.757224,0.444452,0.753969,-0.582505,0.761838,bpr_ndcg
3,16144,0.672244,-0.958536,-0.005133,-0.093083,0.118219,-0.325690,1.434977,0.006304,-0.780396,...,1.159395,-0.746610,0.042197,0.038217,-0.813577,1.753534,-0.063353,-0.829087,-1.122440,most_popular_ndcg
4,18064,-0.813108,0.897909,-0.105261,0.080410,0.099298,-1.109625,2.775797,0.139941,-0.745728,...,0.584199,0.619601,0.027197,0.051431,-1.668557,1.609979,-1.117613,1.222671,-2.145877,lmf_ndcg
